##**VISUALIZATION**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd "/content/drive/My Drive/NYC Taxi"

/content/drive/My Drive/NYC Taxi


In [3]:
ls

'Abhijith_NYC_Taxi(3).ipynb'   tabular_data_jan.parquet          yellow_tripdata_2023-02.parquet
 Abhijith_NYC_Taxi.ipynb       tabular_data_jun.parquet          yellow_tripdata_2023-03.parquet
 NYC.ipynb                     tabular_data_mar.parquet          yellow_tripdata_2023-04.parquet
 tabular_data_apr.parquet      tabular_data_may.parquet          yellow_tripdata_2023-05.parquet
 tabular_data_feb.parquet      ts_data_final.parquet             yellow_tripdata_2023-06.parquet
 tabular_data_final.parquet    yellow_tripdata_2023-01.parquet


In [4]:
import pandas as pd
from datetime import timedelta
import plotly.express as px
from typing import Optional

In [5]:
tabular_data_final = pd.read_parquet('tabular_data_final.parquet')

tabular_data_final

,rides_previous_648_hour,rides_previous_647_hour,rides_previous_646_hour,rides_previous_645_hour,rides_previous_644_hour,rides_previous_643_hour,rides_previous_642_hour,rides_previous_641_hour,rides_previous_640_hour,rides_previous_639_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,19.0,28.0,43.0,33.0,12.0,3.0,2.0,1.0,1.0,1.0,...,3.0,4.0,5.0,3.0,8.0,18.0,22.0,2023-01-28,4,58.0
1,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,2.0,0.0,...,5.0,5.0,7.0,5.0,8.0,15.0,26.0,2023-01-29,4,53.0
2,2.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,2.0,3.0,...,3.0,1.0,5.0,2.0,1.0,2.0,0.0,2023-01-30,4,2.0
3,3.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,6.0,2.0,...,2.0,1.0,1.0,1.0,0.0,0.0,2.0,2023-01-31,4,0.0
4,3.0,16.0,28.0,21.0,12.0,5.0,4.0,2.0,2.0,3.0,...,4.0,2.0,3.0,0.0,5.0,1.0,2.0,2023-01-28,7,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4917,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-06-29,199,0.0
4918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-06-30,199,0.0
4919,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-06-28,245,0.0
4920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-06-29,245,0.0


###Creating a plotting function

In [6]:
def plot_one_sample(
    example_id: int,
    features: pd.DataFrame,
    targets: Optional[pd.Series] = None,
    predictions: Optional[pd.Series] = None
):
    """"""
    features_ = features.iloc[example_id]

    if targets is not None:
        target_ = targets.iloc[example_id]
    else:
        target_ = None

    ts_columns = [c for c in features.columns if c.startswith('rides_previous_')]
    ts_values = [features_[c] for c in ts_columns] + [target_]
    ts_dates = pd.date_range(
        features_['pickup_hour'] - timedelta(hours=len(ts_columns)),
        features_['pickup_hour'],
        freq='H'
    )

    # line plot with past values
    title = f'Pick up hour={features_["pickup_hour"]}, location_id={features_["pickup_location_id"]}'
    fig = px.line(
        x=ts_dates, y=ts_values,
        template='plotly_dark',
        markers=True, title=title
    )

    if targets is not None:
        # green dot for the value we wanna predict
        fig.add_scatter(x=ts_dates[-1:], y=[target_],
                        line_color='green',
                        mode='markers', marker_size=10, name='actual value')

    if predictions is not None:
        # big red X for the predicted value, if passed
        prediction_ = predictions.iloc[example_id]
        fig.add_scatter(x=ts_dates[-1:], y=[prediction_],
                        line_color='red',
                        mode='markers', marker_symbol='x', marker_size=15,
                        name='prediction')
    return fig

In [7]:
features = tabular_data_final.drop(columns=['target_rides_next_hour'])

targets = tabular_data_final['target_rides_next_hour']

In [8]:
plot_one_sample(
    example_id=0,
    features=features,
    targets=targets,
)

In [9]:
plot_one_sample(
    example_id=7,
    features=features,
    targets=targets,
)

In [10]:
plot_one_sample(
    example_id=760,
    features=features,
    targets=targets,
)